In [1]:

import sys
!{sys.executable} -m pip install pandas numpy matplotlib seaborn scikit-learn nltk gensim



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score

from gensim.models import Word2Vec
import pickle
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gultekinqwe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gultekinqwe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# df1 = pd.read_csv("dataFile/spam_modified.csv", sep=';', names=['label', 'text'], usecols=[0, 1])
# df2 = pd.read_csv("dataFile/enron_spam_data_modified.csv", sep=';', names=['label', 'text'], usecols=[0, 1])

# # Boşlukları sil, küçük harfe çevir
# df1['label'] = df1['label'].str.strip().str.lower()
# df2['label'] = df2['label'].str.strip().str.lower()

# # Etiketleri doğru eşle
# df1['label'] = df1['label'].map({'ham': 0, 'spam': 1}).fillna(-1).astype(int)
# df2['label'] = df2['label'].map({'ham': 0, 'spam': 1}).fillna(-1).astype(int)
# df1 = df1[df1["label"] != -1]
# df2 = df2[df2["label"] != -1]


# # NaN varsa temizle
# df1.dropna(inplace=True)
# df2.dropna(inplace=True)

# #birlestirme
# df = pd.concat([df2, df1], ignore_index=True)

# print("Toplam birleşik veri şekli:", df.shape)
# print("Etiket dağılımı:\n", df["label"].value_counts())

# # Son halini gör
# df.head()

df1 = pd.read_csv("dataFile/spam_modified.csv", sep=';', header=0, usecols=[0, 1])
df2 = pd.read_csv("dataFile/enron_spam_data_modified.csv", sep=';', header=0, usecols=[0, 1])

df1.columns = ['label', 'text']
df2.columns = ['label', 'text']

df1['label'] = df1['label'].str.strip().str.lower()
df2['label'] = df2['label'].str.strip().str.lower()

print("Label değerleri:", df2['label'].unique())

df1['label'] = df1['label'].map({'ham': 0, 'spam': 1})
df2['label'] = df2['label'].map({'ham': 0, 'spam': 1})

df1.dropna(subset=['label', 'text'], inplace=True)
df2.dropna(subset=['label', 'text'], inplace=True)

df1['label'] = df1['label'].astype(int)
df2['label'] = df2['label'].astype(int)

df = pd.concat([df1, df2], ignore_index=True)

print("Toplam birleşik veri şekli:", df.shape)
print("Etiket dağılımı:\n", df["label"].value_counts())



Label değerleri: ['ham' 'spam']
Toplam birleşik veri şekli: (38916, 2)
Etiket dağılımı:
 label
0    21317
1    17599
Name: count, dtype: int64


In [4]:
print(df1.shape)
print(df2.shape)

(5571, 2)
(33345, 2)


In [5]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z ]', '', text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return ' '.join(tokens)

df["clean_text"] = df["text"].apply(preprocess)
df.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gultekinqwe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gultekinqwe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,label,text,clean_text
0,0,Ok lar... Joking wif u oni...,ok lar joking wif u oni
1,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
2,0,U dun say so early hor... U c already then say...,u dun say early hor u c already say
3,0,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf life around though
4,1,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darling week word back id like fun...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=3000)
X_tfidf = tfidf.fit_transform(df["clean_text"])

lsa = TruncatedSVD(n_components=300, random_state=42)
X_lsa = lsa.fit_transform(X_tfidf)
X_lsa.shape


(38916, 300)

In [7]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import numpy as np

sentences = [text.split() for text in df["clean_text"]]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2)
word_vectors = w2v_model.wv

kmeans = KMeans(n_clusters=50, random_state=42)
kmeans.fit(word_vectors.vectors)

def get_cluster_features(text):
    cluster_count = np.zeros(50)
    if not isinstance(text, str) or not text.strip():  # Boş veya NaN string varsa
        return cluster_count
    for word in text.split():
        if word in word_vectors:
            try:
                vec = np.asarray(word_vectors[word], dtype=np.float64)
                idx = kmeans.predict([vec])[0]
                cluster_count[idx] += 1
            except Exception as e:
                continue  # hata varsa geç
    return cluster_count


semantic_vectors = np.array([get_cluster_features(text) for text in df["clean_text"]])
semantic_vectors.shape


(38916, 50)

In [8]:
X = np.hstack((X_lsa, semantic_vectors))
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [9]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
svm = SVC(kernel='linear', probability=True)

mlp.fit(X_train, y_train)
rf.fit(X_train, y_train)
svm.fit(X_train, y_train)

# Modelleri kaydet
pickle.dump(mlp, open("models/model_mlp.pkl", "wb"))
pickle.dump(rf, open("models/model_rf.pkl", "wb"))
pickle.dump(svm, open("models/model_svm.pkl", "wb"))


In [10]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score

for model, name in zip([mlp, rf, svm], ["MLP", "Random Forest", "SVM"]):
    y_pred = model.predict(X_test)
    print(f"--- {name} ---")
    print(classification_report(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("ROC AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))
    print()

--- MLP ---
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      4264
           1       0.95      0.96      0.96      3520

    accuracy                           0.96      7784
   macro avg       0.96      0.96      0.96      7784
weighted avg       0.96      0.96      0.96      7784

F1 Score: 0.9559341123468956
Accuracy: 0.9597893114080165
ROC AUC: 0.9941875293151968

--- Random Forest ---
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      4264
           1       0.95      0.94      0.95      3520

    accuracy                           0.95      7784
   macro avg       0.95      0.95      0.95      7784
weighted avg       0.95      0.95      0.95      7784

F1 Score: 0.9460651771737584
Accuracy: 0.9513103802672148
ROC AUC: 0.9901540913354938

--- SVM ---
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      4264
           1     